In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import yfinance as yf
import datetime
    
# using now() to get current time
current_time = datetime.datetime.now()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#fist step is to load the data from yfinance api 

def load_data(ticker,start_date='2020-01-01',end_date=current_time):
    '''
    parameters 
    ticker=stock ticker symbol 
    start_date= date from when to import
    end_date=date till when to import
    '''
    stock_data=yf.download(ticker,start=start_date,end=end_date)
    return stock_data

stock_data=load_data('AAPL','2020-01-01','2025-02-01')

In [ ]:
#now to split the date into training and validation sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
def preprocess_data(stock_data,target_column='Close',test_size=0.2):
    #separate it into features and labels
    X=stock_data.drop([target_column,'Adj Close'], axis=1) #features
    y=stock_data[target_column] #label
    #perform the train valid split
    X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=test_size,random_state=0)
    return X_train, X_valid, y_train, y_valid

X_train, X_valid, y_train, y_valid = preprocess_data(stock_data)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#now lets define the model
def build_model(dropout_ratio,act_func):
#define layers with unit and their activation function,
#add dropout ratio to prevent overfitting
    model= keras.Sequential([
    layers.BatchNormalization(),
    layers.Dense(1024, activation=act_func),
    layers.Dropout(dropout_ratio),
    layers.BatchNormalization(),
    layers.Dense(512, activation=act_func),
    layers.Dropout(dropout_ratio),
    layers.BatchNormalization(),
    layers.Dense(256, activation=act_func),
    layers.Dropout(dropout_ratio),
    layers.BatchNormalization(),    
    layers.Dense(1,activation='linear')
    ])
##select the type of loss and optimizer
    model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='mse',
    metrics=['mean_absolute_error']
    )
    return model

model=build_model(0.3,'relu')

Now to compile and fit the model

In [ ]:
from keras.callbacks import EarlyStopping
#fit the model for training
def train_model(model,X_train, y_train, X_valid, y_valid,epochs=50, batch_size=32):
    history=model.fit(
        x=X_train,
        y= y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose="auto",
        validation_data=(X_valid,y_valid),
    )
    return history

history = train_model(model, X_train, y_train, X_valid, y_valid)

Now to plot the loss land val_loss graphs with accuracy graphs 

In [ ]:
import matplotlib.pyplot as plt
def plot_training_history(history):
    """
    Plots the training history for loss and accuracy.
    
    Parameters:
    history: Training history from model training.
    """
    # Plot loss
    plt.plot(history.history['loss'], label='train_mse')
    plt.plot(history.history['val_loss'], label='val_mse')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('MSE')
    plt.legend()
    plt.show()

    # Plot mean squared error (accuracy)
    plt.plot(history.history['mean_absolute_error'], label='train_mae')
    plt.plot(history.history['val_mean_absolute_error'], label='val_mae')
    plt.title('Model Mean Squared Error')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.legend()
    plt.show()

plot=plot_training_history(history)


Now to predict and check the model using test data 

In [ ]:

def evaluate_model(model, X_test, y_test,current_time):
    """
    Parameters:
    model: Keras model.
    X_test (ndarray): Test features.
    y_test (ndarray): Test labels.
    """
    # import test data
    ticker = "AAPL"
    stock_test = yf.download(ticker, start='2025-02-01', end=current_time)
    X_test=stock_test.drop(['Close','Adj Close'],axis=1)
    y_test=stock_test['Close']
    # Evaluate the model on the test data
    test_mse, test_mae = model.evaluate(X_test, y_test)
    
    print(f"Test Mean Squared Error:",round(test_mse,2))
    print(f"Test Loss:",round(test_mae,2))
    return round(test_mse,2), round(test_mae,2)

test_loss, test_mse = evaluate_model(model, X_test, y_test,current_time)

